# Noteboo for testing JPYModel

In [1]:
%load_ext autoreload
%autoreload 2
import time

In [41]:
from Preprocessing import Preprocessing
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout

In [42]:
usdjpy_prep = Preprocessing().\
        loadDataFromCache(fileName='USDJPY_transformed',
                          labels=['target_1', 'target_2']).\
        divideTrainTest().\
        standarize().\
        createDataset(batchSize=128,
                      prefetch=True).\
        getDataset()


/Users/sebastiansukiennik/Desktop/Tensorflow/MyTensorflow/USDJPY_prediction/Preprocessing.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[mask] = temp_stand
/Users/sebastiansukiennik/Desktop/Tensorflow/MyTensorflow/USDJPY_prediction/Preprocessing.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[mask] = temp_stand
/Users/sebastiansukiennik/Desktop/Tensorflow/MyTensorflow/USDJPY_prediction/Preprocessing.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [43]:
train = usdjpy_prep[0]
validate = usdjpy_prep[1]
test = usdjpy_prep[2]

## Create model

In [44]:
class USDJPYModel(Model):
    
    def __init__(self, input_shape: int=29, **kwargs) -> None:
        """
        Declare layers and their specifications
        """
        super(USDJPYModel, self).__init__(**kwargs)
        self.input_layer = Input(shape=(29, ), name='input_layer')
        self.first_dense = Dense(units=64, activation='relu', name='first_dense')
        self.second_dense = Dense(units=128, activation='swish', name='second_dense')
        self.first_drop = Dropout(0.1, name='first_dropout')
        self.third_dense = Dense(units=128, activation='gelu', name='third_dense')
        self.first_output = Dense(units=1, name='output_1')
        self.second_output = Dense(units=1, activation='sigmoid', name='output2')
    
    def call(self, inputs):
        """
        Connect layers via functional API interface
        """
        x = self.input_layer(inputs)
        x = self.first_dense(x)
        x = self.second_dense(x)
        x = self.first_drop(x)
        x = self.third_dense(x)
        out1 = self.first_output(x)
        out2 = self.second_output(x)

        return out1, out2

In [45]:
model = USDJPYModel(29)
model.compile(
    optimizer='adam',
    loss={'output_1': tf.keras.losses.mean_absolute_error, 'output_2': tf.keras.losses.sparse_categorical_crossentropy},
    metrics=['accuracy']
)

In [47]:
train

<PrefetchDataset element_spec=({'Adj_Close': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Close': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'High': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Low': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Open': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Volume': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Close_sma_20': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Close_sma_50': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Close_sma_800': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Close_sma_120': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Close_sma_150': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Close_ema_20': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Close_ema_50': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Close_ema_800': TensorSpec(shape=(None,), dtype=tf.float64, name=None),

In [26]:
model.fit(train)

TypeError: in user code:

    File "/Users/sebastiansukiennik/.pyenv/versions/3.9.6/envs/USDJPY_prediction/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/sebastiansukiennik/.pyenv/versions/3.9.6/envs/USDJPY_prediction/lib/python3.9/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/sebastiansukiennik/.pyenv/versions/3.9.6/envs/USDJPY_prediction/lib/python3.9/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/sebastiansukiennik/.pyenv/versions/3.9.6/envs/USDJPY_prediction/lib/python3.9/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/Users/sebastiansukiennik/.pyenv/versions/3.9.6/envs/USDJPY_prediction/lib/python3.9/site-packages/keras/ut